In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
import os
from pinecone import Pinecone as pcn
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from IPython.display import Markdown
import gradio as gr
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI 

/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### 01 Get API Keys

In [3]:
api_key = os.environ.get("PINECONE_API_KEY")
openai_api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# configure client
pc = pcn(api_key=api_key)
index_name = 'recipes-index'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1106}},
 'total_vector_count': 1106}

### 02 Set Embedding Model

In [5]:
embed_model = "text-embedding-3-small"
embed = OpenAIEmbeddings(model=embed_model)

In [13]:
text_field = "description"

vectorStore = PineconeVectorStore(
  index, embed, text_field
)

llm = ChatOpenAI(  
    model_name='gpt-4o-mini',  
    temperature=0.0  
)

retriever = vectorStore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
# we can set a similarity score threshold and only return documents with a score above that threshold.
# search_kwargs={"score_threshold": 0.5}
# We can also limit the number of documents k returned by the retriever.
# retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [14]:
def get_recipe(query, history=None):# completion llm  


    # 2. Incorporate the retriever into a question-answering chain.
    system_prompt = (
    """
        Always add a kind and friendly message according to the context at the beginning of the response.
        Take the information from context[0]["metadata"]
        Then follow this format
        Title: [Title of the Recipe]
        Description: [Short Description of the Recipe]
        Preparation Time: [Preparation Time]
        Cooking Time: [Cooking Time]
        Difficulty: [Difficulty Level]
        Serves: [Number of Servings]
        Diet Type: [Diet Type]
        Nutrition: calories context[0]["metadata"]["calories"] | fat context[0]["metadata"]["fat"] | protein context[0]["metadata"]["protein"] | fiber context[0]["metadata"]["fibre"]
        Ingredients: [List of Ingredients]
        Instructions: [Step-by-step Cooking Instructions]

        If you don't have any general information, just respond with "I don't know!"

        {context}
        {{metadata}}
        """
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    response = rag_chain.invoke({"input": query})
    print(response)
    return response["answer"]

## Gradio App

In [15]:
demo = gr.ChatInterface(
  get_recipe,
  title="Recipes Generator",
  description="Ask recipes recommendations based on ingredients and instructions",
  examples=["Recommend a recipe with chicken", "A low-calorie chicken recipe", "What can I do with brocolli"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7884

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.


## Test History

In [20]:
def get_recipe(query, history=True):
    """
    The query can be a string like "chicken recipe". This function will
    return the recipe that matches the query the most.

    Args:
        query (str): The query to search for.
        history (list): A list of dictionaries containing previous interactions.

    Returns:
        str: The recipe that matches the query the most.
    """
    # 1. Create a prompt
    system_prompt = (
        """
        Always add a kind and friendly message according to the context at the beginning of the response.
        Important instruction:
        When the user says phrases like "give me a recipe", "give me something to cook", or any variation of "give me" followed by a recipe request, interpret this as a request to *recommend* a recipe.
        Take the information from context[0]["metadata"]
        Then follow this format:
        Title: [Title of the Recipe]
        Description: [Short Description of the Recipe]
        Preparation Time: [Preparation Time]
        Cooking Time: [Cooking Time]
        Difficulty: [Difficulty Level]
        Serves: [Number of Servings]
        Diet Type: [Diet Type]
        Nutrition: calories context[0]["metadata"]["calories"] | fat context[0]["metadata"]["fat"] | protein context[0]["metadata"]["protein"] | fiber context[0]["metadata"]["fibre"]
        Ingredients: [List of Ingredients]
        Instructions: [Step-by-step Cooking Instructions]

        If you don't have any general information, just respond with "I don't know!"

        {context}
        {{metadata}}
        """
    )

    # 2. Create a chain
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
        ]
    )

    # 3. Add previous history to the prompt if available
    messages = []
    
    if history:
        for item in history:
            role = item["role"]
            content = item["content"]
            messages.append((role, content))
    
    # Add the current query
    messages.append(("human", query))

    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)
    print("#### ragchain", rag_chain)
    print("#### messages", messages)

    # 4. Run the chain with history
    response = rag_chain.invoke({"input": messages})
    print("### resopnse ",response)
    
    # 5. Return the response
    return response["answer"]

demo = gr.ChatInterface(
  get_recipe,
  title="Recipes Generator",
  description="Ask recipes recommendations based on ingredients and instructions",
  examples=["Recommend a recipe with chicken", "A low-calorie chicken recipe", "What can I do with brocolli", "please recommend 3 recipes with rice"],
)

demo.launch()


Running on local URL:  http://127.0.0.1:7888

To create a public link, set `share=True` in `launch()`.


#### ragchain bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x7fc5bcf7e900>, search_kwargs={'k': 3}), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\n        Always add a kind and friendly message according to the context at the beginning of the response.\n        Take the information from context[0]["metadata"]\n        Then follow this format:\n        Title: [Title of the Recipe]\n        Description: [Short 

Traceback (most recent call last):
  File "/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1518, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/gradio/utils.py", line 793, in async_wrapper
    

In [10]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# Inicializando el modelo y el retriever
llm = ChatOpenAI(  
    model_name='gpt-4o-mini',  
    temperature=0.0  
)

retriever = vectorStore.as_retriever(search_type="similarity")

# Prompt para contextualizar la pregunta
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import AIMessage, HumanMessage


def get_recipe(query, history=None):
    # Inicializa el historial de chat si es la primera vez
    history = []    # Prompt del sistema que guía cómo generar la respuesta
    
    system_prompt = (
        """
        Always add a kind and friendly message according to the context at the beginning of the response.
        Take the information from context[0]["metadata"]
        Then follow this format
        Title: [Title of the Recipe]
        Description: [Short Description of the Recipe]
        Preparation Time: [Preparation Time]
        Cooking Time: [Cooking Time]
        Difficulty: [Difficulty Level]
        Serves: [Number of Servings]
        Diet Type: [Diet Type]
        Nutrition: calories context[0]["metadata"]["calories"] | fat context[0]["metadata"]["fat"] | protein context[0]["metadata"]["protein"] | fiber context[0]["metadata"]["fibre"]
        Ingredients: [List of Ingredients]
        Instructions: [Step-by-step Cooking Instructions]

        If you don't have any general information, just respond with "I don't know!"

        {context}
        {{metadata}}
        """
    )

    # Configura el prompt del QA (sistema y humano)
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

    # Crear las cadenas para el QA y el RAG (retrieval-augmented generation)
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    # Invoca la cadena con la pregunta y el historial de chat
    response = rag_chain.invoke({"input": query, "chat_history": history})

    # Actualiza el historial de chat con los nuevos mensajes
    history.extend(
        [
            HumanMessage(content=query),
            AIMessage(content=response["answer"]),
        ]
    )

    # Imprimir el mensaje generado por la IA
    print("####### ai_msg_1:", response)

    # Devolver el contenido del mensaje de la IA
    return response["answer"]

In [17]:
demo = gr.ChatInterface(
  get_recipe,
  title="Recipes Generator",
  description="Ask recipes recommendations based on ingredients and instructions",
  examples=["Recommend a recipe with chicken", "A low-calorie chicken recipe", "What can I do with brocolli", "please recommend 3 recipes with rice"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.
